<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte3Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
%%writefile crearArchivo.py
nuevo_archivo = open("archivo","w")
for i in range(100):
  nuevo_archivo.write(str(i+1) + " Hola, como estan todos?\n")

Writing crearArchivo.py


In [129]:
!python crearArchivo.py 

In [206]:
%%writefile copiarArchivo.py
from multiprocessing import Pipe
import os
import sys

#COMIENZA EL MAIN
#VERIFICO QUE SE HAYA ENVIADO EL NOMBRE DEL ARCHIVO Y OPCIONALMENTE LA CANT BYTES A TRANSFERIR POR CADA ENVIO
if len(sys.argv) < 2 or len(sys.argv) > 3: 
  sys.exit("Paramatros invalidos. Debe ingresar: \nComo minimo, el nombre del archivo a copiar. \nComo maximo, el nombe del archivo a copiar y luego la cantidad de bytes a transferir en cada envío.")

#GUARDO EL NOMBRE DEL ARCHIVO A COPIAR
path = sys.argv[1]

#VERIFICO QUE EL ARCHIVO EXISTE
if not os.path.isfile(path):
   sys.exit("El archivo no existe!")  

#CANT BYTES A TRANSFERIR POR DEFECTO
chunk = 512
   
#SI SE ENVIO UN SEGUNDO PARAMETRO LO VALIDA   
if len(sys.argv) == 3:
  if sys.argv[2].isnumeric():
    chunk = int(sys.argv[2])
  else:
    sys.exit("Paramatro invalido: \nEl segundo parametro, correspondiente a la cantidad de bytes a transferir en cada envío, debe ser numerico.")

#CREO LA TUBERIA
recepcion, envio = Pipe( False )

#CREO PROCESO HIJO
pid = os.fork()

#SI HUBO UN ERROR EN LA CREACION
if pid < 0:
  print("Error al crear nuevo proceso")
  sys.exit(1)

#SI ESTOY EN EL PADRE CIERRO RECEPCION Y LLAMO A ENVIA ARCHIVO
if pid:  
  recepcion.close()
  with open(path,"r") as f:
    while True:
      datos_bytes = (bytes(f.read(chunk), 'utf-8'))
      if not datos_bytes:
        break      
      envio.send_bytes(datos_bytes)       
  envio.close()
  os.wait()
  
#SI ESTOY EN EL HIJO CIERRO ENVIO Y LLAMO A LEER ARCHIVO
else:  
  envio.close()
  new_path = path + "_copia"
  with open(new_path,"w") as f:
    continuar = True
    while continuar:
      try:
        f.write(recepcion.recv_bytes(chunk).decode())            
      except EOFError as e:
        continuar = False       
  recepcion.close()
  os._exit(0)

Overwriting copiarArchivo.py


In [199]:
!python copiarArchivo.py archivo 1024

In [200]:
!python copiarArchivo.py archivo 512

In [209]:
!python copiarArchivo.py archivo 

In [211]:
!python copiarArchivo.py archivo 1024 extra

Paramatros invalidos. Debe ingresar: 
Como minimo, el nombre del archivo a copiar. 
Como maximo, el nombe del archivo a copiar y luego la cantidad de bytes a transferir en cada envío.


In [207]:
!python copiarArchivo.py archivo gdsg

Paramatro invalido: 
El segundo parametro, correspondiente a la cantidad de bytes a transferir en cada envío, debe ser numerico.


In [210]:
!diff archivo archivo_copia